In [1]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

import scipy.optimize

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import seaborn as sns

In [2]:
# general code 

# get time series (ts)
# do smoothing function (ts_smooth)
# subtract (ts - ts_smooth)
# derivative d/dt(ts - ts_smooth), and second deriv d/dt(ts - ts_smooth)
# look at distributions, calculate standard deviations
# calculate accuracy of guesses 

In [13]:
def take_derivative(series):
    return pd.Series(h.normalize(np.gradient(series)))

def plot_many(fig, time, series, names, ):
    for name, ser in zip(names, series):
            fig.add_trace(go.Scatter(x=time, y=ser, name=name, mode='lines'))
    return fig

In [39]:
class Window_Detect2:
    def __init__(self, df):
        self.time = df["DateTime"]
        # magnitude of time passed in seconds 
        time_mag = self.time - self.time.min()
        self.time_seconds = time_mag.dt.total_seconds()

        self.window = df["Window Open"]
        self.window_norm = h.normalize(self.window)

        self.temp = df["Temp C"]
        self.temp_norm = h.normalize(self.temp)


    def analyze_window_change(self, smooth_fx):
        self.smooth_series = h.normalize(smooth_fx(self.temp_norm, self.time_seconds))
        # TODO: mean largest - mean smallest?
        self.dif = h.normalize(self.temp_norm - self.smooth_series)
        self.deriv = take_derivative(self.dif)
        self.deriv2 = take_derivative(self.deriv)
        self.std, self.std2 = self.deriv.std(), self.deriv2.std()
        self.zscore, self.zscore2 = h.calc_zscore(self.deriv), h.calc_zscore(self.deriv2)


    def plot_analysis(self):
        fig = go.Figure()

        series = [self.window_norm, self.temp_norm, self.smooth_series, self.dif, self.deriv, self.deriv2]
        names = ["Window", "Observed", "Smoothed", "Difference", "Deriv1", "Deriv2"]

        return plot_many(fig, self.time, series, names)


    def plot_distributions(self, marker_width=0.1, bin_size=0.003):
        print(f"Std 1 = {self.std}, Std 2 = {self.std2}")
        fig = go.Figure()

        for ix, ser in enumerate([self.deriv2, self.deriv]):
            opacity = 0.9 if ix == 0 else 1
            fig.add_trace(go.Histogram(
            x=ser, histnorm='probability', name=f' Deriv{2 - ix}', opacity=opacity, marker_line=dict(width=marker_width ,color='black'), xbins=dict( size=bin_size),))

        fig.update_layout(barmode="stack")

        return fig
        

    def plot_zscore(self):
        fig = go.Figure()

        series = [self.window_norm, self.zscore, self.zscore2]
        names = ["Window", "Z-Score 1", "Z-Score 2"]

        return plot_many(fig, self.time, series, names)


    def plot_guesses(self):
        self.guess_mask = (w1.zscore2 > 2) | (w1.zscore2 <= -2)
        self.guess_values = self.deriv[mask]
        self.guess_times = self.time[mask]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.time, y=self.window_norm, name="Window", mode='lines'))
        fig.add_trace(go.Scatter(x=self.guess_times, y=self.guess_values, name="Guess", mode='markers'))

        return fig


    def plot_analysis_and_distributions(self):
        # TODO make into subplots fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
        fig1 = self.plot_analysis()
        fig2 = self.plot_distributions()
        fig1.show()
        fig2.show()



In [8]:
def make_stl_smooth(series, time, model="additive"):
    n_samples = 4 
    seasonality_period = 24
    # 15 minute intervals (4/hour), 24 hour periodicitity
    period = n_samples*seasonality_period
    result = seasonal_decompose(series, model=model, period=period)
    return result.seasonal

def make_sin_smooth(series, time):
    fit = h.fit_sin(time, series)
    return fit["fitfunc"](time)

def make_ewm_smooth(series, time, level=4):
    return series.ewm(level).mean()



In [6]:
a00, a01 = h.import_desired_data("A", "15T")
print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

[1.] [0. 1.] Index(['DateTime', 'Temp C', 'RH %', 'Room', 'Ambient Temp', 'Ambient RH',
       'Window Open'],
      dtype='object')


In [40]:
w1 = Window_Detect2(a01)
w1.analyze_window_change(make_stl_smooth)
# w1.plot_zscore()
w1.plot_guesses()
# w1.plot_analysis_and_distributions()

In [ ]:
mask = (w1.zscore2 > 2) | (w1.zscore2 <= -2)
m = w1.deriv.loc[mask]
w1.time[mask]

In [41]:
w1 = Window_Detect2(a01)
w1.analyze_window_change(make_sin_smooth)

# w1.plot_zscore()
w1.plot_guesses()
# w1.plot_analysis_and_distributions()

In [42]:
w1 = Window_Detect2(a01)
w1.analyze_window_change(make_ewm_smooth)
# w1.plot_zscore()
w1.plot_guesses()
# w1.plot_analysis_and_distributions()